In [18]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
import utils.featurizer as ft
import utils.encodeador as le

##### Read all csv's

In [20]:
test = pd.read_csv("data/test.csv")

In [21]:
df_test = pd.read_csv("data/test.csv")

In [22]:
train = pd.read_csv("data/train.csv")

#### Use featizer in train.csv

In [23]:
train_ft = ft.featurizer(train)

In [24]:
train_ft_enc = le.label_encoder(train_ft)

In [25]:
drop_cols = ['fecha', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng', 'id']

In [26]:
X = train_ft_enc.drop(['precio'] + drop_cols, axis =1 )
y = train_ft_enc.precio

In [27]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [28]:
some_param = {'bootstrap': True,
 'max_depth': 70,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 10,
 'n_estimators': 400
             }

In [29]:
randomForest = RandomForestRegressor(**some_param)

In [30]:
%%time
randomForest.fit(X_train,y_train)

CPU times: user 7min 4s, sys: 301 ms, total: 7min 4s
Wall time: 7min 4s


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=70,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=4, min_samples_split=10,
                      min_weight_fraction_leaf=0.0, n_estimators=400,
                      n_jobs=None, oob_score=False, random_state=None,
                      verbose=0, warm_start=False)

In [31]:
def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)
    errors = abs(predictions - test_labels)
    mape = 100 * np.mean(errors / test_labels)
    accuracy = 100 - mape
    print('Model Performance')
    print('Average Error: {:0.4f} degrees.'.format(np.mean(errors)))
    print('Accuracy = {:0.2f}%.'.format(accuracy))
    print('MAE = {}'.format(mean_absolute_error(test_labels,predictions)))
    
    return accuracy



In [32]:
%%time
base_accuracy = evaluate(randomForest, X_test, y_test)

Model Performance
Average Error: 617339.0564 degrees.
Accuracy = 69.60%.
MAE = 617339.0564132711
CPU times: user 5.3 s, sys: 4.01 ms, total: 5.3 s
Wall time: 5.3 s


In [33]:
feature_importances = pd.DataFrame(randomForest.feature_importances_,
                                   index = X_train.columns,
                                    columns=['importance']).sort_values('importance', ascending=False)

In [34]:
feature_importances

,importance
metroscubiertos,0.397658
is_DF,0.096313
tipodepropiedad,0.091351
metrostotales,0.078855
ciudad,0.061660
provincia,0.052680
banos,0.051001
year,0.029196
antiguedad,0.025016
metro_promedio_por_cuadricula,0.022288


In [35]:
df_submit_test = ft.featurizer(test)

In [36]:
df_submit_test = le.label_encoder(df_submit_test)

In [37]:
drop_cols = ['fecha', 'idzona', 'titulo', 'descripcion', 'direccion', 'lat', 'lng', 'id']

In [38]:
df_submit_test = df_submit_test.drop( drop_cols, axis = 1)

In [39]:
pred = randomForest.predict(df_submit_test)

In [41]:
res = pd.DataFrame(pred, index=df_test.id, columns=['target'])
display(res.head())
res.to_csv("data/workshop-randomforest-v2_1.csv", header=True)

,target
id,
4941,5.339665e+06
51775,1.231381e+06
115253,2.605897e+06
299321,1.361539e+06
173570,5.999641e+05
